In [ ]:
#importing libraries
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler


In [ ]:

def Get_Clean_Combined_Dataset():

    df = pd.read_csv("tcd-ml-1920-group-income-train.csv")
    df
    
    df.drop_duplicates(subset ="Instance", 
                     keep = False, inplace = True)
    df2 = pd.read_csv("tcd-ml-1920-group-income-test.csv")
    df2["Instance"] = df2["Instance"] + 991709
    
    data = pd.concat([df,df2], axis=0)
    data = data.reset_index(drop=True)
    data['Yearly Income in addition to Salary (e.g. Rental Income)'] = data['Yearly Income in addition to Salary (e.g. Rental Income)'].str.replace(' EUR', '').astype(float)
    return data

In [ ]:


data_set = Get_Clean_Combined_Dataset()
data_set["Total Yearly Income [EUR]"] = data_set["Total Yearly Income [EUR]"] - data_set["Yearly Income in addition to Salary (e.g. Rental Income)"]


In [ ]:

#Split data into income set (Incuding instance so that the unlabled data can extracted later)
y = data_set[["Instance","Total Yearly Income [EUR]","Yearly Income in addition to Salary (e.g. Rental Income)"]].copy()

#And all the Feature data (Incuding instance so that the unlabled data can extracted later)
X = data_set[["Instance","Year of Record","Age"]].copy()
X.head()

In [ ]:
#Function to relpace invalid entries in numeric features with the mean walue of the particular feature
def ReplaceNan_Numeric(df,F_Name):
    average = df[F_Name].dropna().mean(axis=0)
    df[F_Name] = df[F_Name].replace(np.nan, average, inplace=False)
    return df

In [ ]:

def MinMaxScale_Feature(df, F_Name):
    feature_array = df[F_Name].to_numpy()
    feature_array = feature_array.reshape(-1, 1)
    
    scaler = MinMaxScaler()
    scaled_array = scaler.fit_transform(feature_array)
    
    scaled_df = pd.DataFrame(scaled_array)
    df = df.drop(F_Name,1)   #Feature Matrix
    new_df = pd.concat([df, scaled_df], axis=1)
    return new_df

In [ ]:
# Clean numeric data
X = ReplaceNan_Numeric(X,"Year of Record")
X = ReplaceNan_Numeric(X,"Age")


In [ ]:
# Scale numeric data
X = MinMaxScale_Feature(X,"Year of Record")
X = MinMaxScale_Feature(X,"Age")


In [ ]:
#Split the data back out to separate sets on key 'Instance'
df2_X = X.loc[X["Instance"] > 991709]
X = X.loc[X["Instance"] <= 991709]         
df2_y = y.loc[y["Instance"] > 991709]
y = y.loc[y["Instance"] <= 991709]

#Drop 'Instance' from datasets now that it is no longer needed as a key
df2_X = df2_X.drop("Instance",1)
X = X.drop("Instance",1)
y = y.drop("Instance",1)

In [ ]:
# Split the test and training data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

add_income = y_test["Yearly Income in addition to Salary (e.g. Rental Income)"].values

y_train = y_train.drop("Yearly Income in addition to Salary (e.g. Rental Income)",1)
y_test = y_test.drop("Yearly Income in addition to Salary (e.g. Rental Income)",1)


In [ ]:


# Create ridgecv regression object with 4 crossvalidation folds
regr = RidgeCV(alphas=np.array([0.0001,0.001,0.01,0.1,1]),fit_intercept=True,normalize=False,cv = 4)

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)



In [ ]:
add_income = add_income.reshape(-1,1)
y_pred = np.add(y_pred,add_income)


In [ ]:
# Print results
print("Mean absolute error: %.2f"
      % mean_absolute_error(y_test, y_pred))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))

In [ ]:
# Make predictions on the unlabled dataset
df2_y_pred = regr.predict(df2_X)
# export data
pd.DataFrame(df2_y_pred).to_csv("Predictions.csv")